In [7]:
import sys
import yaml
import torch
import logging
from pathlib import Path

# Add the path to the directory containing the omnicell package
# Assuming the omnicell package is in the parent directory of your notebook
sys.path.append('..')  # Adjust this path as needed

import yaml
import torch
import logging
from pathlib import Path
from omnicell.config.config import Config, ETLConfig, ModelConfig, DatasplitConfig, EvalConfig, EmbeddingConfig
from omnicell.data.loader import DataLoader
from omnicell.constants import PERT_KEY, GENE_EMBEDDING_KEY, CONTROL_PERT, OMNICELL_ROOT, SYNTHETIC_DATA_PATHS_KEY
from omnicell.models.selector import load_model

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configure paths
MODEL_CONFIG = ModelConfig.from_yaml(f"{OMNICELL_ROOT}/configs/models/scot/proportional_scot.yaml")
ETL_CONFIG = ETLConfig.from_yaml(f"{OMNICELL_ROOT}/configs/ETL/no_preproc_drop_unmatched_scot.yaml")
EMBEDDING_CONFIG = EmbeddingConfig(pert_embedding='GenePT')

SPLIT_CONFIG = DatasplitConfig.from_yaml(f"{OMNICELL_ROOT}/configs/splits/repogle_k562_essential_raw/random_splits/rs_accP_k562_ood_ss:ns_20_2_most_pert_0.1/split_0/split_config.yaml")
EVAL_CONFIG = EvalConfig.from_yaml(f"{OMNICELL_ROOT}/configs/splits/repogle_k562_essential_raw/random_splits/rs_accP_k562_ood_ss:ns_20_2_most_pert_0.1/split_0/eval_config.yaml")  # Set this if you want to run evaluations

# Load configurations
config = Config(model_config=MODEL_CONFIG,
                 etl_config=ETL_CONFIG, 
                 datasplit_config=SPLIT_CONFIG, 
                 eval_config=EVAL_CONFIG)




#Alternatively you can initialize the config objects manually as follows:
# etl_config = ETLConfig(name = XXX, log1p = False, drop_unmatched_perts = False, ...)
# model_config = ...
# embedding_config = ...
# datasplit_config = ...
# eval_config = ...
# config = Config(etl_config, model_config, datasplit_config, eval_config)

# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize data loader and load training data
loader = DataLoader(config)
adata, pert_rep_map = loader.get_training_data()

# Get dimensions and perturbation IDs
input_dim = adata.X.shape[1]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pert_ids = adata.obs[PERT_KEY].unique()
gene_emb_dim = adata.varm[GENE_EMBEDDING_KEY].shape[1] if GENE_EMBEDDING_KEY in adata.varm else None

print(f"Data loaded:")
print(f"- Number of cells: {adata.shape[0]}")
print(f"- Input dimension: {input_dim}")
print(f"- Number of perturbations: {len(pert_ids)}")



2025-02-17 13:05:35,510 - INFO - Loading data catalogue from /orcd/data/omarabu/001/opitcho/omnicell/configs/catalogue/FrangiehIzar2021_protein.yaml
2025-02-17 13:05:35,512 - INFO - Loading data catalogue from /orcd/data/omarabu/001/opitcho/omnicell/configs/catalogue/PapalexiSatija2021_eccite_arrayed_RNA.yaml
2025-02-17 13:05:35,514 - INFO - Loading data catalogue from /orcd/data/omarabu/001/opitcho/omnicell/configs/catalogue/AdamsonWeissman2016_GSM2406677_10X005.yaml
2025-02-17 13:05:35,515 - INFO - Loading data catalogue from /orcd/data/omarabu/001/opitcho/omnicell/configs/catalogue/PapalexiSatija2021_eccite_protein.yaml
2025-02-17 13:05:35,516 - INFO - Loading data catalogue from /orcd/data/omarabu/001/opitcho/omnicell/configs/catalogue/SchraivogelSteinmetz2020_TAP_SCREEN__chromosome_11_screen.yaml
2025-02-17 13:05:35,518 - INFO - Loading data catalogue from /orcd/data/omarabu/001/opitcho/omnicell/configs/catalogue/DatlingerBock2021.yaml
2025-02-17 13:05:35,519 - INFO - Loading data

Using device: cuda


2025-02-17 13:05:39,937 - INFO - Loaded unpreprocessed data, # of data points: 310385, # of genes: 8563.
2025-02-17 13:05:39,939 - INFO - Preprocessing training data
2025-02-17 13:05:39,941 - INFO - Using identity features for perturbations
2025-02-17 13:05:40,053 - INFO - Removing observations with perturbations not in the dataset as a column
2025-02-17 13:05:40,246 - INFO - Removed 189 perturbations that were not in the dataset columns and 0 perturbations that did not have an embedding for a total of 189 perturbations removed out of an initial 2058 perturbations
2025-02-17 13:06:11,688 - INFO - Doing OOD split


Data loaded:
- Number of cells: 279630
- Input dimension: 8563
- Number of perturbations: 1850


In [8]:

adata.uns[SYNTHETIC_DATA_PATHS_KEY]

[PosixPath('/orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/synthetic_data/proportional_scot_no_preproc_drop_unmatched_b924ee39/synthetic_counterfactuals_14.pkl'),
 PosixPath('/orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/synthetic_data/proportional_scot_no_preproc_drop_unmatched_b924ee39/synthetic_counterfactuals_13.pkl'),
 PosixPath('/orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/synthetic_data/proportional_scot_no_preproc_drop_unmatched_b924ee39/synthetic_counterfactuals_8.pkl'),
 PosixPath('/orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/synthetic_data/proportional_scot_no_preproc_drop_unmatched_b924ee39/synthetic_counterfactuals_1.pkl'),
 PosixPath('/orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/synthetic_data/proportional_scot_no_preproc_drop_unmatched_b924ee39/synthetic_counterfactuals_6.pkl'),
 PosixPath('/orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/syn